<a href="https://colab.research.google.com/github/afontoura/DRLA2ALM/blob/master/stable_baselines_TD3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install stable_baselines==2.9.0
!git clone https://github.com/afontoura/DRLA2ALM
%cd DRLA2ALM
!pip install -e .

In [0]:
import gym
import envs
import numpy as np

from stable_baselines import TD3
from stable_baselines.td3.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.ddpg.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

time_horizon = 80
discount_rate = .08
current_seed = 10

env = gym.make('ALM-v0', T = time_horizon, rate = discount_rate)

In [0]:
# The noise objects for TD3
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = TD3(MlpPolicy, env, action_noise=action_noise, verbose=1, seed = current_seed)
model.learn(total_timesteps=500000, log_interval=100)

In [0]:
resultados = []
for i in np.arange(1000):
  dones = False
  G = []
  obs = env.reset()
  while not dones:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    G.append(rewards)
  resultados.append(np.array(G).sum())
  #print(str(i) + ': ' + str(np.array(G).sum()))

In [0]:
print((np.array(resultados) == 80).sum())
print(np.array(resultados).mean())

603
55.746


In [0]:
model.save('td3_t' + str(time_horizon) + '_dr' + str(round(discount_rate * 1000)) + '_seed' + str(current_seed))